<a href="https://colab.research.google.com/github/Lococo231/MO_Tasks/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install pandas seaborn scikit-learn

# Домашнее задание 3

1. Выбрать номинальное значение как целевую переменную
2. Провести разведочный анализ **включая** анализ зависимости между номинальными переменными (chi2)
3. Выбрать `best_features`
4. Сделать нормализацию данных (scaler, onehotencoding)
4. Обучить модель логистической регрессии
5. Вывести метрики качества, описать полученный результат (также вывести коэфициенты модели)

In [45]:

import numpy as np
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)


df = pd.read_csv('dataset.csv')

df = df.drop(columns=['Blood Pressure'])

categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

print(categorical_columns)


['Gender', 'Occupation', 'BMI Category', 'Sleep Disorder']


In [46]:

print(f"Размер: {df.shape}")
print(f"\nТипы данных:\n{df.dtypes}")
print(f"\nПропущенные значения:\n{df.isnull().sum()}")

categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()

print(f"НОМИНАЛЬНЫЕ ПЕРЕМЕННЫЕ: {categorical_columns}")
print(f"ЧИСЛОВЫЕ ПЕРЕМЕННЫЕ: {numeric_columns}")

print("ОПИСАТЕЛЬНАЯ СТАТИСТИКА (числовые)")
print(df[numeric_columns].describe())

print("РАСПРЕДЕЛЕНИЕ НОМИНАЛЬНЫХ ПЕРЕМЕННЫХ")
for col in categorical_columns:
    print(f"\n{col}:")
    print(df[col].value_counts())

print("АНАЛИЗ ЗАВИСИМОСТИ МЕЖДУ НОМИНАЛЬНЫМИ ПЕРЕМЕННЫМИ (CHI-2)")
for i, col1 in enumerate(categorical_columns):
    for col2 in categorical_columns[i+1:]:
        contingency_table = pd.crosstab(df[col1], df[col2])
        chi2, p_value, dof, expected = chi2_contingency(contingency_table)
        cramers_v = np.sqrt(chi2 / (len(df) * (min(contingency_table.shape) - 1)))

        print(f"\n{col1} vs {col2}:")
        print(f"Chi-2 статистика: {chi2:.4f}")
        print(f"P-value: {p_value:.2e}")
        print(f"Cramér's V: {cramers_v:.4f}")

        if cramers_v > 0.5:
            strength = "СИЛЬНАЯ"
        elif cramers_v > 0.3:
            strength = "СРЕДНЯЯ"
        elif cramers_v > 0.1:
            strength = "СЛАБАЯ"
        else:
            strength = "ОЧЕНЬ СЛАБАЯ"

        print(f"Сила связи: {strength}")

#Chi-2 — насколько фактические данные отличаются от ожидаемых. Ищем, чтобы обнаружить связь.
#P-value — вероятность ошибки. P < 0.05 = связь значимая.
#Cramér's V — сила связи (0-1). Ищем, чтобы узнать, насколько сильно переменные связаны.

Размер: (374, 12)

Типы данных:
Person ID                    int64
Gender                      object
Age                          int64
Occupation                  object
Sleep Duration             float64
Quality of Sleep             int64
Physical Activity Level      int64
Stress Level                 int64
BMI Category                object
Heart Rate                   int64
Daily Steps                  int64
Sleep Disorder              object
dtype: object

Пропущенные значения:
Person ID                    0
Gender                       0
Age                          0
Occupation                   0
Sleep Duration               0
Quality of Sleep             0
Physical Activity Level      0
Stress Level                 0
BMI Category                 0
Heart Rate                   0
Daily Steps                  0
Sleep Disorder             219
dtype: int64
НОМИНАЛЬНЫЕ ПЕРЕМЕННЫЕ: ['Gender', 'Occupation', 'BMI Category', 'Sleep Disorder']
ЧИСЛОВЫЕ ПЕРЕМЕННЫЕ: ['Person ID', 'Age', '

### EDA

In [47]:

# y = gender
# = other best

In [57]:


best_features = ['Gender', 'Occupation']
numeric_features = ['Age', 'Sleep Duration', 'Quality of Sleep', 'Physical Activity Level',
                    'Stress Level', 'Heart Rate', 'Daily Steps']

# Масштабирование численных признаков
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(df[numeric_features]), columns=numeric_features)

# Получаем категориальные признаки (без Gender, так как он будет целевой переменной)
df_categorical = df[['Occupation']].reset_index(drop=True)

# One-hot encoding для категориальных признаков
ohe = OneHotEncoder(sparse_output=False, drop=None)
df_categorical_encoded = pd.DataFrame(
    ohe.fit_transform(df_categorical),
    columns=ohe.get_feature_names_out(['Occupation'])
)

# Объединяем все признаки (численные + закодированные категориальные)
#X = pd.concat([df_numeric.reset_index(drop=True),
#               df_categorical_encoded.reset_index(drop=True)], axis=1)

print("Выбранные признаки из категориальных ", best_features)
print("Кодированные колонки:")
print(X.columns.tolist())
X

Выбранные признаки из категориальных  ['Gender', 'Occupation']
Кодированные колонки:
['Age', 'Sleep Duration', 'Quality of Sleep', 'Physical Activity Level', 'Stress Level', 'Heart Rate', 'Daily Steps']


,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps
0,-1.753096,-1.298887,-1.098280,-0.825418,0.347021,1.654719,-1.619584
1,-1.637643,-1.173036,-1.098280,0.039844,1.475592,1.170474,1.970077
2,-1.637643,-1.173036,-1.098280,0.039844,1.475592,1.170474,1.970077
3,-1.637643,-1.550588,-2.771424,-1.402260,1.475592,3.591698,-2.362273
4,-1.637643,-1.550588,-2.771424,-1.402260,1.475592,3.591698,-2.362273
...,...,...,...,...,...,...,...
369,1.941401,1.218127,1.411435,0.760896,-1.345836,-0.524383,0.113356
370,1.941401,1.092276,1.411435,0.760896,-1.345836,-0.524383,0.113356
371,1.941401,1.218127,1.411435,0.760896,-1.345836,-0.524383,0.113356
372,1.941401,1.218127,1.411435,0.760896,-1.345836,-0.524383,0.113356


In [58]:
y = df['Gender']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

model = LogisticRegression(random_state=42, max_iter=1000, multi_class='multinomial')
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

y_train_proba = model.predict_proba(X_train)
y_test_proba = model.predict_proba(X_test)

print("ОБУЧАЮЩАЯ ВЫБОРКА")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
print(f"Precision (weighted): {precision_score(y_train, y_train_pred, average='weighted'):.4f}")
print(f"Recall (weighted): {recall_score(y_train, y_train_pred, average='weighted'):.4f}")
print(f"F1 (weighted): {f1_score(y_train, y_train_pred, average='weighted'):.4f}")

print("\nТЕСТОВАЯ ВЫБОРКА")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")
print(f"Precision (weighted): {precision_score(y_test, y_test_pred, average='weighted'):.4f}")
print(f"Recall (weighted): {recall_score(y_test, y_test_pred, average='weighted'):.4f}")
print(f"F1 (weighted): {f1_score(y_test, y_test_pred, average='weighted'):.4f}")

print("\nМАТРИЦА ОШИБОК (тестовая выборка):")
print(confusion_matrix(y_test, y_test_pred))

print("\nКЛАССИФИКАЦИОННЫЙ ОТЧЕТ:")
print(classification_report(y_test, y_test_pred,
                          target_names=['Female', 'Male']))

#Accuracy — доля правильных предсказаний из всех. Сколько % мы угадали.
#Precision — из всех, что мы предсказали как "положительный класс", сколько действительно положительные (не ошибок).
#Recall — из всех действительно положительных, сколько мы нашли (не пропустили).
#F1 — это гармоническое среднее Precision и Recall.

#Когда классы сбалансированы и модель работает симметрично, эти значения стремятся к совпадению.

ОБУЧАЮЩАЯ ВЫБОРКА
Accuracy: 0.8889
Precision (weighted): 0.8901
Recall (weighted): 0.8889
F1 (weighted): 0.8888

ТЕСТОВАЯ ВЫБОРКА
Accuracy: 0.8673
Precision (weighted): 0.8684
Recall (weighted): 0.8673
F1 (weighted): 0.8672

МАТРИЦА ОШИБОК (тестовая выборка):
[[50  6]
 [ 9 48]]

КЛАССИФИКАЦИОННЫЙ ОТЧЕТ:
              precision    recall  f1-score   support

      Female       0.85      0.89      0.87        56
        Male       0.89      0.84      0.86        57

    accuracy                           0.87       113
   macro avg       0.87      0.87      0.87       113
weighted avg       0.87      0.87      0.87       113



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


ОБУЧАЮЩАЯ ВЫБОРКА
Accuracy: 0.9847
Precision (weighted): 0.9847
Recall (weighted): 0.9847
F1 (weighted): 0.9847

ТЕСТОВАЯ ВЫБОРКА
Accuracy: 0.9558
Precision (weighted): 0.9570
Recall (weighted): 0.9558
F1 (weighted): 0.9557

МАТРИЦА ОШИБОК (тестовая выборка):
[[52  4]
 [ 1 56]]

КЛАССИФИКАЦИОННЫЙ ОТЧЕТ:
              precision    recall  f1-score   support
      Female       0.98      0.93      0.95        56
        Male       0.93      0.98      0.96        57
    accuracy                           0.96       113
   macro avg       0.96      0.96      0.96       113
weighted avg       0.96      0.96      0.96       113


In [61]:

print(classification_report(y_test, y_test_pred, target_names=['Female', 'Male']))

print("КОЭФФИЦИЕНТЫ МОДЕЛИ")

coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_[0],
})

coef_df = coef_df.sort_values('Coefficient', ascending=False)

print(coef_df.to_string(index=False))

print("\nПерехват (Intercept):", model.intercept_[0])

#Если Accuracy близко к 1 - модель хорошо предсказывает
#Если разница между обучением и тестом > 0.1 - переучивание
#F1 показывает баланс между Precision и Recall
#Коэффициенты > 0 увеличивают вероятность класса, < 0 уменьшают

              precision    recall  f1-score   support

      Female       0.85      0.89      0.87        56
        Male       0.89      0.84      0.86        57

    accuracy                           0.87       113
   macro avg       0.87      0.87      0.87       113
weighted avg       0.87      0.87      0.87       113

КОЭФФИЦИЕНТЫ МОДЕЛИ
                Feature  Coefficient
           Stress Level     1.735393
         Sleep Duration     1.186058
Physical Activity Level     0.290950
       Quality of Sleep     0.273137
             Heart Rate    -0.084608
            Daily Steps    -0.386028
                    Age    -1.072837

Перехват (Intercept): 0.07150752018197176


### Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
y = df['your_feature']
X = df[best_features]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
) # stratify - разделение данных с учетом распределнеия категориальной переменной

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)


In [ ]:
...